In [1]:
!pip install --upgrade --no-cache-dir gdown >/dev/null
!gdown 1kTJmHH3qEyivtSJvnVUMYpWnZcQl5XaV

Downloading...
From: https://drive.google.com/uc?id=1kTJmHH3qEyivtSJvnVUMYpWnZcQl5XaV
To: /content/Bigdata_hw2_datasets.zip
100% 6.28M/6.28M [00:00<00:00, 46.4MB/s]


In [2]:
!unzip Bigdata_hw2_datasets.zip

Archive:  Bigdata_hw2_datasets.zip
   creating: Bigdata_hw2_datasets/
   creating: Bigdata_hw2_datasets/q1/
  inflating: Bigdata_hw2_datasets/q1/stream_data_dgim.txt  
   creating: Bigdata_hw2_datasets/q2/
  inflating: Bigdata_hw2_datasets/q2/games.csv  
  inflating: Bigdata_hw2_datasets/q2/ratings.csv  
   creating: Bigdata_hw2_datasets/q3/
  inflating: Bigdata_hw2_datasets/q3/c1.txt  
  inflating: Bigdata_hw2_datasets/q3/c2.txt  
  inflating: Bigdata_hw2_datasets/q3/data.txt  


In [3]:
import math
import numpy as np
import time

INPUT_FILE = 'Bigdata_hw2_datasets/q1/stream_data_dgim.txt'

In [125]:
class DGIM:
    def __init__(self, file, N):
        """
        Implementation of DGIM Algorithm

        Args:
            file (str): Input file path
            N (int): Window size
        """
        self.file = file
        self.N = N
        self.ts = 0
        self.buckets = []
        self._init_buckets()

    def _init_buckets(self):
        """
        Create the buckets according to the window size.
        """
        for i in range(int(math.log(self.N, 2))+1):
            self.buckets.append([])

    def _old_bucket(self, k=0):
        """
        Find the oldest bucket according to k.

        Args:
            k (int, optional): Defaults to N.

        Returns:
            tuple: old bucket number size and old bucket end-timestamp 
        """
        k = self.N if k == 0 else k
        obi = 0
        obt = 0
        for i in range(len(self.buckets)):
            for ets in self.buckets[i]:
                if ets >= self.ts - k:
                    obi = i
                    obt = ets
                else:
                    return obi, obt
        return obi, obt

    def _update(self):
        """
        Update the buckets based on the algorithm constraints.
        If we have more than 2 buckets of each size, merge them.
        """
        for i in range(len(self.buckets)):
            if len(self.buckets[i]) > 2:
                self.buckets[i].pop()
                tmp = self.buckets[i].pop()
                if i != len(self.buckets) - 1:
                    self.buckets[i+1].insert(0, tmp)

    def count(self, k=0):
        """
        Count the ones in the last k bits.

        Args:
            k (int, optional): Defaults to N.

        Returns:
            int: count
        """
        cnt = 0
        obi, obt = self._old_bucket(k)
        for i in range(len(self.buckets)):
            if i > obi:
                break
            for ets in self.buckets[i]:
                if ets > obt:
                    cnt += 2**i
                elif ets == obt:
                    cnt += int(0.5 * 2**i)
        return cnt

    def run(self):
        """
        Read the file in the streaming mode and process.
        """
        with open(self.file) as f:
            while True:
                x = f.read(2).strip()
                if not x:
                    break
                self.ts += 1
                obi, obt = self._old_bucket()
                if obt == self.ts - self.N:
                    self.buckets[obi].remove(obt)
                if x == "1":
                    self.buckets[0].insert(0, self.ts)
                    self._update()

In [126]:
start = time.time()

dgim = DGIM(INPUT_FILE, 1000)
dgim.run()
result_1000 = dgim.count(1000)
 
time1 = time.time() - start

print("The number of 1 bits in the window with size 1000:", result_1000)
print("The running time of DGIM with window size 1000:", time1)

The number of 1 bits in the window with size 1000: 508
The running time of DGIM with window size 1000: 0.19089221954345703


In [128]:
print("The number of 1 bits in the last 500 bits:", dgim.count(500))
print("The number of 1 bits in the last 200 bits:", dgim.count(200))

The number of 1 bits in the last 500 bits: 220
The number of 1 bits in the last 200 bits: 76


In [129]:
start = time.time()

data = np.loadtxt(INPUT_FILE, dtype="uint8")
acc_cnt = np.sum(data[data.shape[0] - 1000:])

time2 = time.time() - start

print("The accurate number of 1 bits in the window with size 1000:", acc_cnt)
print("The running time of accurate result with window size 1000:", time2)
print("The accuracy of DGIM with window size 1000:", 1 - abs(result_1000 - acc_cnt) / acc_cnt)

The accurate number of 1 bits in the window with size 1000: 391
The running time of accurate result with window size 1000: 0.03008556365966797
The accuracy of DGIM with window size 1000: 0.70076726342711
